In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\krsid\tensorflow_datasets\imdb_reviews\plain_text\incomplete.5NS4I9_1.0.0\imdb_reviews-trai…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\krsid\tensorflow_datasets\imdb_reviews\plain_text\incomplete.5NS4I9_1.0.0\imdb_reviews-test…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\krsid\tensorflow_datasets\imdb_reviews\plain_text\incomplete.5NS4I9_1.0.0\imdb_reviews-unsu…

Dataset imdb_reviews downloaded and prepared to C:\Users\krsid\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [3]:
# Tokenize and pad the reviews
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
train_sentences, train_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in train_data])
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')

test_sentences, test_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in test_data])
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [6]:
# RNN Model

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 64),
    keras.layers.SimpleRNN(64),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.5088 - loss: 0.6959 - val_accuracy: 0.5264 - val_loss: 0.6910
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.6136 - loss: 0.6559 - val_accuracy: 0.5178 - val_loss: 0.7002
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.6922 - loss: 0.5896 - val_accuracy: 0.5382 - val_loss: 0.7694
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.7913 - loss: 0.4396 - val_accuracy: 0.5488 - val_loss: 0.8268
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.8424 - loss: 0.3438 - val_accuracy: 0.5260 - val_loss: 0.9618
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8849 - loss: 0.2615 - val_accuracy: 0.5278 - val_loss: 1.1210
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9194 - loss: 0.1836 - val_accuracy: 0.5335 - val_loss: 1.3335
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9423 - loss: 0.1254 - 

In [8]:
# LSTM

from keras.layers import Bidirectional, LSTM

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32),
    Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop])

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 35ms/step - accuracy: 0.5852 - loss: 0.6888 - val_accuracy: 0.7847 - val_loss: 0.4949
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8015 - loss: 0.4727 - val_accuracy: 0.7957 - val_loss: 0.4593
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step - accuracy: 0.8345 - loss: 0.4099 - val_accuracy: 0.7914 - val_loss: 0.4639
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 34ms/step - accuracy: 0.8522 - loss: 0.3750 - val_accuracy: 0.7838 - val_loss: 0.4789
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8630 - loss: 0.3501 - val_accuracy: 0.7855 - val_loss: 0.4759


In [9]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)

In [15]:
from keras.layers import Conv1D, MaxPooling1D

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32),
    Conv1D(32, 7, activation='relu'),
    MaxPooling1D(5),
    Bidirectional(LSTM(32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint = keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop, checkpoint])

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.5806 - loss: 0.7490 - val_accuracy: 0.8000 - val_loss: 0.4408
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8346 - loss: 0.3962 - val_accuracy: 0.8045 - val_loss: 0.4305
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8667 - loss: 0.3364 - val_accuracy: 0.7880 - val_loss: 0.4790
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8849 - loss: 0.3024 - val_accuracy: 0.8146 - val_loss: 0.4347
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9014 - loss: 0.2707 - val_accuracy: 0.8124 - val_loss: 0.4539
